In [1]:
import webregParser as wrp
from webregScheduler import *
import pandas as pd
from datetime import datetime

In [2]:
schedule = wrp.webregParser("webregMain.pdf")

In [3]:
weekly_df = schedule.get_weekly()
non_weekly_df = schedule.get_non_weekly()

In [4]:
weekly_df

,Subject Course,Title,Section Code,Type,Days,Time,Instructor,Units,Status / (Position),BLDG,Room,Grade Option,Node Title
0,DSC 80,Practice of Data Science,A00,LE,"[Tu, Th]","[3:30p, 4:50p]","Lau, Samuel Ethan",4.0,Enrolled,CENTR,216,L,DSC 80 LE
1,DSC 80,DSC 80 DI,A01,DI,[F],"[10:00a, 10:50a]",NaN,NaN,NaN,CENTR,212,NaN,DSC 80 DI
2,JAPN 10A,First Year Japanese I,D00,LE,"[Tu, Th]","[9:30a, 10:50a]","Iwamoto, Naoki",5.0,Enrolled,CENTR,207,L,JAPN 10A LE
3,JAPN 10A,JAPN 10A TU,D01,TU,"[M, W, F]","[9:00a, 9:50a]",NaN,NaN,NaN,CENTR,202,NaN,JAPN 10A TU
4,MATH 20D,Intro/Differential Equations,B00,LE,"[M, W, F]","[11:00a, 11:50a]","Ohm, Ko Woon",4.0,Enrolled,PCYNH,106,L,MATH 20D LE
5,MATH 20D,MATH 20D DI,B01,DI,[Th],"[6:00p, 6:50p]",NaN,NaN,NaN,APM,6402,NaN,MATH 20D DI
6,MATH 140A,Foundations of Real Analysis I,B00,LE,"[M, W, F]","[2:00p, 2:50p]","Sheng, Hongyi",4.0,Enrolled,WLH,2111,L,MATH 140A LE
7,MATH 140A,MATH 140A DI,B01,DI,[Tu],"[6:00p, 6:50p]",NaN,NaN,NaN,PODEM,0133,NaN,MATH 140A DI
8,MATH 173A,Optimization/Data Science I,A00,LE,"[Tu, Th]","[2:00p, 3:20p]","Cloninger, Alexander",4.0,Planned,CENTR,119,L,MATH 173A LE
9,MATH 173A,MATH 173A DI,A02,DI,[F],"[6:00p, 6:50p]",NaN,NaN,NaN,APM,5402,NaN,MATH 173A DI


In [5]:
def reformat_time(time):
    # Add 0 to the front of the time if it is missing
    if time[1] == ":":
        time = "0" + time
    # Replace "a" with "AM" and "p" with "PM"
    if time[-1] == "a":
        time = time[:-1] + "AM"
    elif time[-1] == "p":
        time = time[:-1] + "PM"
    return time

In [6]:
weekly_df["Start Time"] = weekly_df["Time"].apply(lambda x: reformat_time(x[0]))
weekly_df["End Time"] = weekly_df["Time"].apply(lambda x: reformat_time(x[1]))
weekly_df["Location"] = weekly_df["BLDG"] + " " + weekly_df["Room"]
formatted_wdf = weekly_df[["Node Title", "Days", "Start Time", "End Time", "Location"]]
formatted_wdf = formatted_wdf.fillna("—")
formatted_wdf = formatted_wdf.explode("Days")
formatted_wdf = formatted_wdf.reset_index(drop=True)

In [7]:
formatted_wdf


,Node Title,Days,Start Time,End Time,Location
0,DSC 80 LE,Tu,03:30PM,04:50PM,CENTR 216
1,DSC 80 LE,Th,03:30PM,04:50PM,CENTR 216
2,DSC 80 DI,F,10:00AM,10:50AM,CENTR 212
3,JAPN 10A LE,Tu,09:30AM,10:50AM,CENTR 207
4,JAPN 10A LE,Th,09:30AM,10:50AM,CENTR 207
5,JAPN 10A TU,M,09:00AM,09:50AM,CENTR 202
6,JAPN 10A TU,W,09:00AM,09:50AM,CENTR 202
7,JAPN 10A TU,F,09:00AM,09:50AM,CENTR 202
8,MATH 20D LE,M,11:00AM,11:50AM,PCYNH 106
9,MATH 20D LE,W,11:00AM,11:50AM,PCYNH 106


In [8]:
days = ["M", "Tu", "W", "Th", "F"]
weekly_schedule = {}

[weekly_schedule.update({day: DaySchedule(day)}) for day in days]

print("Weekly Schedule initialized")
for entry in formatted_wdf.values:
    day = entry[1]
    new_node = CourseNode(*entry)
    weekly_schedule[day].insert(new_node)

Weekly Schedule initialized


In [9]:
weekly_schedule

{'M': M Schedule: JAPN 10A TU, MATH 20D LE, MATH 140A LE,
 'Tu': Tu Schedule: JAPN 10A LE, MATH 173A LE, DSC 80 LE, MATH 140A DI,
 'W': W Schedule: JAPN 10A TU, MATH 20D LE, MATH 140A LE,
 'Th': Th Schedule: JAPN 10A LE, MATH 173A LE, DSC 80 LE, MATH 20D DI,
 'F': F Schedule: JAPN 10A TU, DSC 80 DI, MATH 20D LE, MATH 140A LE, MATH 173A DI}